# deeplodocus-engine

Шахматный движок, который проводит первичную фильтрацию ходов с помощью минимакса с альфа-бета отсеченьем и выбирает лучший ход при помощи обученной MLP модели.


<p style="text-align:center;"><img src="media/logo.png" alt="drawing" width="200"/></p>

## Stockfish
### Чтобы иметь возможность посоревноваться с движком

In [1]:
from __future__ import division

import os
import time
import random

import socket
import json

import chess
import chess.pgn
import chess.uci
import stockfish

import numpy as np
from keras.models import load_model

# https://stockfishchess.org/download/
stockfishEngine = stockfish.Stockfish(path="/home/sergey/stockfish/stockfish-ubuntu-x86-64-avx2", depth=8)

/tmp/ipykernel_51515/2465506089.py:12: DeprecationWarning: The chess.uci module is deprecated in favor of
chess.engine <https://python-chess.readthedocs.io/en/latest/engine.html>.

Please consider updating and open an issue
<https://github.com/niklasf/python-chess/issues/new> if your use case
is not covered by the new API.
  import chess.uci
2023-09-06 15:55:14.941963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Deeplodocus model
### Импортируем обученную модель

In [2]:
model = load_model('700k_open.h5') # На подходе 1_400_000_ep400.h5

## Преобразование доски в bitmap

<p style="text-align:center;"><img src="media/board.jpg" alt="drawing" width="500"/></p>

In [18]:
def splitter(inputStr, black):
    inputStr = format(inputStr, "064b")
    tmp = [inputStr[i:i+8] for i in range(0, len(inputStr), 8)]
    for i in range(0, len(tmp)):
        tmp2 = list(tmp[i])
        tmp2 = [int(x) * black for x in tmp2]
        tmp[i] = tmp2

    return tmp

def get_bitmap(board):
    bitmap = []
    
    P_input = splitter(int(board.pieces(chess.PAWN, chess.WHITE)), 1)
    R_input = splitter(int(board.pieces(chess.ROOK, chess.WHITE)), 1)
    N_input = splitter(int(board.pieces(chess.KNIGHT, chess.WHITE)), 1)
    B_input = splitter(int(board.pieces(chess.BISHOP, chess.WHITE)), 1)
    Q_input = splitter(int(board.pieces(chess.QUEEN, chess.WHITE)), 1)
    K_input = splitter(int(board.pieces(chess.KING, chess.WHITE)), 1)

    p_input = splitter(int(board.pieces(chess.PAWN, chess.BLACK)), -1)
    r_input = splitter(int(board.pieces(chess.ROOK, chess.BLACK)), -1)
    n_input = splitter(int(board.pieces(chess.KNIGHT, chess.BLACK)), -1)
    b_input = splitter(int(board.pieces(chess.BISHOP, chess.BLACK)), -1)
    q_input = splitter(int(board.pieces(chess.QUEEN, chess.BLACK)), -1)
    k_input = splitter(int(board.pieces(chess.KING, chess.BLACK)), -1)

    bitmap.extend(P_input)
    bitmap.extend(R_input)
    bitmap.extend(N_input)
    bitmap.extend(B_input)
    bitmap.extend(Q_input)
    bitmap.extend(K_input)
    bitmap.extend(p_input)
    bitmap.extend(r_input)
    bitmap.extend(n_input)
    bitmap.extend(b_input)
    bitmap.extend(q_input)
    bitmap.extend(k_input)

    return np.array([bitmap]).ravel()

## Примитивная функция оценки для минимакса

In [39]:
def count_est_for_minimax(board, main_color, current_color):
    piece_value = {chess.PAWN:10, chess.KNIGHT:30,chess.BISHOP:30,chess.ROOK:50, chess.QUEEN:90}
    evaluation = 0
    for piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN]:
        evaluation += len(board.pieces(piece_type, main_color)) * piece_value.get(piece_type)
    
    return evaluation

def position_evaluation_for_minimax(board, main_color, current_color):
    return count_est_for_minimax(board, main_color,current_color) - \
            count_est_for_minimax(board, not main_color, current_color);

## Алгоритм минимакса

In [40]:
def minimax(deep, max_deep, board, current_color, last_move, main_color, possible_moves):
    if deep == 0 or board.is_game_over():
        return [last_move, position_evaluation_for_minimax(board, main_color, not current_color)]
    
    is_max = current_color == main_color
    v = float("-inf") if is_max else float("inf")
    
    move_and_est = [[None], v]
    
    for move in possible_moves:
        board.push(move)
        recursive_value = minimax(deep - 1, max_deep, board, not current_color, move, \
                                    main_color, board.legal_moves)
        board.pop()
        
        if is_max:
            if move_and_est[1] < recursive_value[1]:
                move_and_est = [[move], recursive_value[1]]
            elif deep == max_deep and move_and_est[1] == recursive_value[1]:
                move_and_est[0].append(move)
        else:
            if move_and_est[1] > recursive_value[1]:
                move_and_est = [[move], recursive_value[1]]
            elif deep == max_deep and move_and_est[1] == recursive_value[1]:
                move_and_est[0].append(move)
        
    return move_and_est

## Выберем при помощи MLP лучший ход, среди предложенных минимаксом

In [41]:
def get_best_move_by_deeplodocus(fen):
    start_time = time.time()
    
    board = chess.Board()
    board.set_fen(fen)
    
    deep = 3
    best_moves_ = minimax(deep, deep, board, board.turn, None, board.turn, board.legal_moves)[0]
    
    print(len(best_moves_))
    
    max_ = -2
    res_move = 0
    
    if not board.turn:
        max_ = 2
    
    for move in best_moves_:
        board.push(move)
        bitmap = get_bitmap(board)
        pred = model.predict(np.array([bitmap]))[0][0]
        
        if not board.turn and pred > max_:
            max_ = pred
            res_move = move
        elif board.turn and pred < max_:
            max_ = pred
            res_move = move
        
        board.pop()
    
    print("--- %s seconds ---" % (time.time() - start_time))
    return res_move.uci()

## Deeplodocus_native

### Возможность поиграть с deeplodocus без минимакса

In [42]:
def simple_evaluate_best_move(board):
    max_ = -2
    res_move = 0
    
    if not board.turn:
        max_ = 2
    
    for move in board.legal_moves:
        board.push(move)
        bitmap = get_bitmap(board)
        pred = model.predict(np.array([bitmap]))[0][0]
        
        if not board.turn and pred > max_:
            max_ = pred
            res_move = move
        elif board.turn and pred < max_:
            max_ = pred
            res_move = move
        
        board.pop()
    
    return res_move.uci()

def get_best_move_by_deeplodocus_native(fen):
    board = chess.Board()
    board.set_fen(fen)
    
    return simple_evaluate_best_move(board)

## Dto

In [43]:
class Dto:
    class ServerResponse:
        def __init__(self, move):
            self.move = move

    class ServerRequest:
        def __init__(self, fen, algorithm):
            self.fen = fen
            self.algorithm = algorithm

## AiRunner

Класс для запуска определённой модели по её имени

In [44]:
class AiRunner:
    @staticmethod
    def run(server_request):   
        if str(server_request.algorithm) == "STOCKFISH":   
            stockfishEngine.set_fen_position(server_request.fen)
            return Dto.ServerResponse(stockfishEngine.get_best_move())
        elif str(server_request.algorithm) == "DEEPLODOCUS_NATIVE":
            return Dto.ServerResponse(get_best_move_by_deeplodocus_native(server_request.fen))
        elif str(server_request.algorithm) == "DEEPLODOCUS":
            return Dto.ServerResponse(get_best_move_by_deeplodocus(server_request.fen))

## Server

Класс для запуска локального сервера


In [45]:
class Server:
    HOSTNAME = "localhost"
    PORT = 6671

    def __init__(self):
        self.server = socket.socket()
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    def run(self):
        try:
            self.server.bind((self.HOSTNAME, self.PORT))
            self.server.listen()

            print("server starts")
            while True:
                con, addr = self.server.accept()
                json_str = con.recv(1024).decode('utf-8').replace('\n', '')

                print("server get data:", json_str)
                temp_obj = json.loads(json_str)

                server_req = Dto.ServerRequest(**temp_obj)
                server_res = AiRunner.run(server_req)

                message = json.dumps(server_res.__dict__)
                con.send(message.encode())

                print("server send message:", message)
                con.close()
        finally:
            self.server.close()

## Запуск сервера

In [38]:
Server().run()

server starts
server get data: {"fen":"rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1","algorithm":"DEEPLODOCUS"}
18
1/1 [==============================] - 0s 31ms/step
--- 2.0251541137695312 seconds ---
server send message: {"move": "b8c6"}
server get data: {"fen":"r1bqkbnr/pppppppp/2n5/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 2 2","algorithm":"DEEPLODOCUS"}
2
1/1 [==============================] - 0s 27ms/step
--- 0.7332379817962646 seconds ---
server send message: {"move": "g8f6"}
server get data: {"fen":"r1bqkb1r/pppppppp/2n2n2/8/4P3/2N2N2/PPPP1PPP/R1BQKB1R b KQkq - 4 3","algorithm":"DEEPLODOCUS"}
13
1/1 [==============================] - 0s 26ms/step
--- 1.778956651687622 seconds ---
server send message: {"move": "g7g6"}
server get data: {"fen":"r1bqkb1r/pppppp1p/2n2np1/8/3PP3/2N2N2/PPP2PPP/R1BQKB1R b KQkq d3 0 4","algorithm":"DEEPLODOCUS"}
12
1/1 [==============================] - 0s 27ms/step
--- 1.9262580871582031 seconds ---
server send message: {"move": "f8g7"}
serve

1/1 [==============================] - 0s 33ms/step
--- 3.259427070617676 seconds ---
server send message: {"move": "c8f5"}
server get data: {"fen":"r4rk1/pp1Nppbp/5np1/1B1P1b2/8/2N4P/1qPB1PP1/3QR1K1 b - - 2 18","algorithm":"DEEPLODOCUS"}
2
1/1 [==============================] - 0s 30ms/step
--- 2.9037232398986816 seconds ---
server send message: {"move": "f5d7"}
server get data: {"fen":"r4rk1/pp1Bppbp/5np1/3P4/8/2N4P/1qPB1PP1/3QR1K1 b - - 0 19","algorithm":"DEEPLODOCUS"}
1
1/1 [==============================] - 0s 36ms/step
--- 2.1387484073638916 seconds ---
server send message: {"move": "f6d7"}
server get data: {"fen":"r4rk1/pp1nRpbp/6p1/3P4/8/2N4P/1qPB1PP1/3Q2K1 b - - 0 20","algorithm":"DEEPLODOCUS"}
1
1/1 [==============================] - 0s 33ms/step
--- 2.239605665206909 seconds ---
server send message: {"move": "g7c3"}
server get data: {"fen":"r4rk1/pp1nRp1p/6p1/3P4/8/2B4P/1qP2PP1/3Q2K1 b - - 0 21","algorithm":"DEEPLODOCUS"}
1
1/1 [==============================] - 0s 29ms/step

server get data: {"fen":"r2Q1rk1/p4p1p/6p1/4q3/8/2pQ2PP/5P1K/8 b - - 0 32","algorithm":"DEEPLODOCUS"}
2
1/1 [==============================] - 0s 29ms/step
--- 1.886852741241455 seconds ---
server send message: {"move": "a8d8"}
server get data: {"fen":"3Q1rk1/p4p1p/6p1/4q3/8/2p3PP/5P1K/8 b - - 0 33","algorithm":"DEEPLODOCUS"}
1
1/1 [==============================] - 0s 25ms/step
--- 0.9753620624542236 seconds ---
server send message: {"move": "f8d8"}
server get data: {"fen":"3r2k1/p4p1p/6p1/4q3/5P2/2p3PP/7K/8 b - f3 0 34","algorithm":"DEEPLODOCUS"}
10
1/1 [==============================] - 0s 31ms/step
--- 1.2144496440887451 seconds ---
server send message: {"move": "d8d2"}
server get data: {"fen":"6k1/p4p1p/6p1/4q3/5P2/2p3PP/3r4/7K b - - 2 35","algorithm":"DEEPLODOCUS"}
10
1/1 [==============================] - 0s 33ms/step
--- 1.1442694664001465 seconds ---
server send message: {"move": "e5d4"}
server get data: {"fen":"6k1/p4p1p/6p1/8/3q1P1P/2p3P1/3r4/7K b - - 0 36","algorithm":"DEEP

KeyboardInterrupt: 

## Lichess bot